In [3]:
import os
import yaml
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from building_energy_storage_simulation import Environment
from stable_baselines3 import PPO, DDPG
from stable_baselines3.common.monitor import Monitor
import pandas as pd
import matplotlib.pyplot as plt
import pickle 
import torch
import torch.nn as nn
from torch.distributions import Categorical
from torch import softmax

ModuleNotFoundError: No module named 'building_energy_storage_simulation'

In [ ]:
# Define the path to the saved model and logs
logs_path = "logs/ppo/1686578523"

# Load environment configuration from file
environment_config_path = os.path.join("configs", "env.yaml")
with open(environment_config_path, "r") as f:
    environment_config = yaml.safe_load(f)

# Initialize the environment with test dataset
env = Environment(dataset="test.csv", **environment_config)

# Wrap the environment with Monitor for logging and DummyVecEnv for normalization
env = Monitor(env, filename=None)
env = DummyVecEnv([lambda: env])
env = VecNormalize.load(os.path.join(logs_path, "env.pkl"), env)

# Load the trained model
model = PPO.load(os.path.join(logs_path, "model"), env=env)

In [ ]:
#VPG 
class NNModel(nn.Module):
    def __init__(self, input_dim, output_dim): 
        super(NNModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 20)
    
    def forward(self, x): 
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return softmax(self.fc3(x), dim=1)
    
    def predict(self, x):
        with torch.no_grad():
            output = self.forward(x)
            action = Categorical(output).sample().item()
            if(action <= 10): 
                action *= -1
            else: 
                action -= 10
            action *= 0.1
            return torch.tensor([[action]])


In [ ]:
#vpg 
# Define the path to the saved model and logs
logs_path = "logs/vpg/1686578902"

# Load environment configuration from file
environment_config_path = os.path.join("configs", "env.yaml")
with open(environment_config_path, "r") as f:
    environment_config = yaml.safe_load(f)

# Initialize the environment with test dataset
env = Environment(dataset="test.csv", **environment_config)

# Wrap the environment with Monitor for logging and DummyVecEnv for normalization
env = Monitor(env, filename=None)
env = DummyVecEnv([lambda: env])
env = VecNormalize.load(os.path.join(logs_path, "env.pkl"), env)

# Load the trained model
model = NNModel(16, 20)
model.load_state_dict(torch.load(os.path.join(logs_path, "model.pth")))

In [ ]:
env.training = False

actions, observations, electricity_consumption, excess_energy, cost_of_external_generator, rewards = ([], [], [], [], [], [])
done = False
obs = env.reset()
while not done:
        action = model.predict(torch.as_tensor(obs, dtype=torch.float32))
        obs, r, done, info = env.step([float(action.detach())])
        actions.append(action.item())
        original_reward = env.get_original_reward()[0]
        original_obs = env.get_original_obs()[0]
        observations.append(original_obs)
        electricity_consumption.append(info[0]['electricity_consumption'])
        excess_energy.append(info[0]['excess_energy'])
        cost_of_external_generator.append(info[0]['cost_of_external_generator'])
        rewards.append(r)

trajectory = pd.DataFrame({
        'action': actions,
        'observations': observations,
        'electricity_consumption': electricity_consumption,
        'excess_energy': excess_energy,
        'cost_of_external_generator': cost_of_external_generator,
        'reward': rewards
    })

trajectory.to_csv(os.path.join(logs_path, "test_results.csv"), index=False)

In [ ]:
def plot_scores(scores, rolling_window=100):
    """Plot scores and optional rolling mean using specified window."""
    plt.plot(scores); plt.title("Scores");
    rolling_mean = pd.Series(scores).rolling(rolling_window).mean()
    plt.plot(rolling_mean, c='r');
    return rolling_mean

rolling_mean = plot_scores(trajectory['reward'])

In [ ]:
trajectory

In [ ]:
day_number = 0
plot_data = trajectory[day_number * 24 : (day_number + 3) * 24]
observation_df = plot_data['observations'].apply(pd.Series)

plt.rcParams["figure.figsize"] = (16,4)

fig, ax = plt.subplots()
ax.plot(observation_df[0], label = 'state of charge')
ax.plot(observation_df[1], label = 'electric load')
ax.plot(observation_df[6], label = 'solar generation')
ax.plot(observation_df[11], label = 'external generator cost')
ax.plot(plot_data['excess_energy'], label = 'excess energy')
ax.plot(plot_data['electricity_consumption'], label = 'electricity consumption')

ax1 = ax.twinx()
ax1.plot(plot_data['action'], label = 'action', color = 'black')
fig.legend(bbox_to_anchor=[0.5, 0.95], loc = 'center', ncol=5, prop={'size': 16})

In [ ]:
env.training = False

costs = []
done = False
obs = env.reset()
while not done:
    action = model.predict(torch.as_tensor(obs, dtype=torch.float32))
    obs, r, done, info = env.step([float(action.detach())])
    costs.append(info[0]['cost_of_external_generator'])

total_costs = sum(costs)

In [ ]:
env.training = False

baseline_costs = []
done = False
obs = env.reset()
while not done:
    # Always taking noop as action. This is the electricity demand if there would be no battery
    action = [0]
    obs, r, done, info = env.step(action)
    baseline_costs.append(info[0]['cost_of_external_generator'])

baseline_total_costs = sum(baseline_costs)

In [ ]:
money_saved = baseline_total_costs - total_costs
percentage_saved = money_saved / baseline_total_costs * 100
print(f'Money saved: {money_saved} $')
print(f'Percentage saved: {percentage_saved} %')